In [1]:
import glob
import json
import os

import cv2

# local .json file with local path info
geladas_root = os.path.dirname(os.getcwd())
local_paths_file = os.path.join(geladas_root, 'local-paths.json')
with open(local_paths_file, "r") as json_file:
    local_paths = json.load(json_file)

import sys
sys.path.append(local_paths['overhead_functions_path'])

from TrackCorrectionGUI import TrackCorrectionGUI

In [2]:
#SETUP

# The yellow track is the focal track that is being corrected
# The blue track is the other track that might be merged with 
# the focal track

# KEY FUNCTIONS:
# next frame = "]"
# previous frame = "["
# advance 3 frames = ";"
# go back 3 frames = "k"
# advance 30 frames = "x"
# go back 30 frames = "z"
# advance 90 frames = "9"
# go back 90 frames = "0"
# advance 5000 frames = "1"
# go back 5000 frames = "2"

# go to frame at beginning of current possible merge track = "b"
# go to frame at end of yellow (focal) track = "g"
# next possible merge track = "o"
# previous possible merge track = "l"
# next focal track = "." (same button as '>')
# previous focal track = "," (same button as '<')
# add point to focal track = click anywhere on picture
# merge focal track with current selected merge track = space bar
# permanently remove blue track = "-" 

# split focal track at current frame = "s"
# split possible merge track at current frame = "a"

# undo last action = "u"
# hide tracks (so easier to see frame) = "h"

In [6]:
#THINGS FOR USER TO ADD:
#file containing info about the tracks

# The tracks file should be a list of dictionaries
# Each dictionary contains information about one track
# The dictionary must have the key "track", "first_frame", "last_frame"
# the key "track" returns an numpy array of shape (N, 2) where N is the length of the track in frames
# each point in the track is in numpy array indexing so row, column from top left corner of image

# The key "first_frame" returns the first frame number of the video the track appears
# The key "last_frame" returns the last frame number of the video that the track appears
video_name = "DJI_0205"

tracks_file = os.path.join(local_paths['base_gelada'], "processing", video_name, 
                           "tracks-complete.npy")

# Path to frames
frames_folder = os.path.join(local_paths['video_folder'], "frames", video_name)
frame_files = sorted(glob.glob(os.path.join(frames_folder, '*.jpg')))
 
# how much you want to rescale the original frame resolution to display (and fit)on the screen    
factor = 0.9

In [7]:
import numpy as np

In [8]:
#RUN
print('starting')
#get good track, draw image on window, set mouse to work
window = TrackCorrectionGUI(tracks_file, frame_files, factor, point_scale=1)
print('initialized')
window.find_next_track()
print('found track')
window.draw_window()
print('drawing_window')
cv2.namedWindow('pic0')
cv2.setMouseCallback('pic0', window.clicked)

#loop to keep image updating
while True:
    # exit when user presses esc key
    key = cv2.waitKey(2) & 0xff
    window.detect_keys(key)
    if key == 27:
        window.save('active')
        break
cv2.destroyAllWindows()

starting
initialized
found track
drawing_window
saved at /mount/zfs-kn/gelada_project/processing/DJI_0205/tracks-complete-0.0.npy
saved at /mount/zfs-kn/gelada_project/processing/DJI_0205/tracks-complete-final.npy
